# Notes, Tips and Tricks

## 🌲 1) Approximate Depth of a Decision Tree

* A well-balanced binary tree with $m$ **leaves** has a depth of approximately:

  $$
  \lceil \log_2(m)^2 \rceil
  $$

* In a **Decision Tree** trained **without restrictions**, there is typically **one leaf per training instance**.

* So if the training set has $10^6$ instances (1 million):

  $$
  \log_2(10^6) \approx 20
  $$

> 📌 Thus, the tree will likely have a depth around **20**.

---

## 📉 2) Is a Node’s Gini Impurity Lower Than Its Parent?

* **Yes, usually.**
  A node's Gini impurity is generally **lower than its parent's**.

* The **CART algorithm** (used in Decision Trees) chooses splits to **minimize the weighted sum** of the children’s impurities.

* 🔄 However:

  * It's **possible** for one child node to have **higher impurity** than the parent.
  * This is allowed **if the other child has sufficiently lower impurity** to offset it.

---

## 🧮 3) Computational Complexity of Decision Trees

* Training complexity:

  $$
  \mathcal{O}(n \times m \log m)
  $$

  Where:

  * $n$ = number of features
  * $m$ = number of training instances

* If you increase the dataset size 10×, the **expected time multiplier** is:

  $$
  K = \frac{n \times 10m \times \log(10m)}{n \times m \times \log(m)} = 10 \times \frac{\log(10m)}{\log(m)}
  $$

* Example with $m = 10^6$:

  $$
  K \approx 10 \times \frac{\log(10^7)}{\log(10^6)} = 10 \times \frac{7}{6} \approx 11.7
  $$

> 🕐 So training time increases **about 11.7×** if dataset size increases 10×.

---

## ⚠️ 4) Presorting Training Set with 100,000 Instances

* **Presorting** the data can **speed up training** only for **small datasets** (a few thousand instances or less).

* For **large datasets** like $100{,}000$ instances:

  > Setting `presort=True` will **slow down training significantly**.

> ✅ Best to keep `presort=False` (default in recent versions of scikit-learn).


In [1]:
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.datasets import make_moons
from sklearn.tree import DecisionTreeClassifier
X_moons, y_moons = make_moons(n_samples=1000, noise=0.4, random_state=42)

X_train, X_test, y_train, y_test = train_test_split(X_moons, y_moons, test_size=0.2, random_state=42)

params = {
	'max_leaf_nodes': list(range(2, 100)),
	'min_samples_split': [2, 3, 4]
}
grid_tree_cv = GridSearchCV(DecisionTreeClassifier(random_state=42), param_grid=params, cv=3, verbose=3)

grid_tree_cv.fit(X_train, y_train)

Fitting 3 folds for each of 294 candidates, totalling 882 fits
[CV 1/3] END max_leaf_nodes=2, min_samples_split=2;, score=0.757 total time=   0.0s
[CV 2/3] END max_leaf_nodes=2, min_samples_split=2;, score=0.772 total time=   0.0s
[CV 3/3] END max_leaf_nodes=2, min_samples_split=2;, score=0.774 total time=   0.0s
[CV 1/3] END max_leaf_nodes=2, min_samples_split=3;, score=0.757 total time=   0.0s
[CV 2/3] END max_leaf_nodes=2, min_samples_split=3;, score=0.772 total time=   0.0s
[CV 3/3] END max_leaf_nodes=2, min_samples_split=3;, score=0.774 total time=   0.0s
[CV 1/3] END max_leaf_nodes=2, min_samples_split=4;, score=0.757 total time=   0.0s
[CV 2/3] END max_leaf_nodes=2, min_samples_split=4;, score=0.772 total time=   0.0s
[CV 3/3] END max_leaf_nodes=2, min_samples_split=4;, score=0.774 total time=   0.0s
[CV 1/3] END max_leaf_nodes=3, min_samples_split=2;, score=0.813 total time=   0.0s
[CV 2/3] END max_leaf_nodes=3, min_samples_split=2;, score=0.820 total time=   0.0s
[CV 3/3] END 

,estimator,DecisionTreeC...ndom_state=42)
,param_grid,"{'max_leaf_nodes': [2, 3, ...], 'min_samples_split': [2, 3, ...]}"
,scoring,None
,n_jobs,None
,refit,True
,cv,3
,verbose,3
,pre_dispatch,'2*n_jobs'
,error_score,nan
,return_train_score,False
,criterion,'gini'


In [2]:
grid_tree_cv.best_estimator_

,criterion,'gini'
,splitter,'best'
,max_depth,None
,min_samples_split,2
,min_samples_leaf,1
,min_weight_fraction_leaf,0.0
,max_features,None
,random_state,42
,max_leaf_nodes,4
,min_impurity_decrease,0.0
,class_weight,None


In [3]:
from sklearn.metrics import accuracy_score

y_pred = grid_tree_cv.predict(X_test)
accuracy_score(y_test, y_pred)

0.855

### Grow a forest

In [6]:
from sklearn.model_selection import ShuffleSplit

n_trees = 1000
n_instances = 100

mini_sets = []
cv = ShuffleSplit(n_splits=n_trees, test_size=(len(X_train) - n_instances), random_state=42)

for mini_train_idx, mini_test_idx in cv.split(X_train):
	X_mini_train = X_train[mini_train_idx]
	y_mini_train = y_train[mini_train_idx]
	mini_sets.append((X_mini_train, y_mini_train))


In [7]:
from sklearn.base import clone
import numpy as np

forest = [clone(grid_tree_cv.best_estimator_) for _ in range(n_trees)]

accuracy_scores = []

for tree, (X_mini_train, y_mini_train) in zip(forest, mini_sets):
	tree.fit(X_mini_train, y_mini_train)

	y_pred = tree.predict(X_test)
	accuracy_scores.append(accuracy_score(y_test, y_pred))

np.mean(accuracy_scores)

np.float64(0.817115)

In [8]:
Y_pred = np.empty([n_trees, len(X_test)], dtype=np.uint8)

for tree_idx, tree in enumerate(forest):
	Y_pred[tree_idx] = tree.predict(X_test)

In [9]:
from scipy.stats import mode

y_pred_majority_votes, n_votes = mode(Y_pred, axis=0)

In [10]:
y_pred_majority_votes

array([1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0,
       1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0,
       0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1,
       1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0,
       0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0,
       1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1,
       1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0,
       1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0,
       0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0,
       1, 0])

In [11]:
accuracy_score(y_test, y_pred_majority_votes.reshape([-1]))

0.85